In [1]:
# Import packages
import numpy as np
import tensorflow as tf
import keras
from keras import Model
import os, glob
import resnet_modified # modified to have zero verbosity, deperacated several printing step, lr -> learning_rate in calling adam.
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from resnet_modified import ResNet
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model
from keras import layers
from keras.layers import Dense
import pandas as pd

In [2]:
(x, y), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)


In [5]:
adv_x_tr_pgd_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_untarget.npy")
adv_x_val_pgd_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_val_untarget.npy")

adv_x_tr_pgd_by_resnet56v1_0_target_to_ll = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_target_to_ll.npy")
adv_x_val_pgd_by_resnet56v1_0_target_to_ll = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_val_target_to_ll.npy")

adv_x_tr_cw_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/cwl2_x_tr_untargeted.npy")
adv_x_val_cw_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/cwl2_x_v_untargeted.npy")



nb_iter = 8
norm = 2

eps = 216/255
eps_iter = 96/255
These are parameters for PGD.
L2 norm, big perturbation (~ 0.83) for pgd

typical parameter for succesful pgd attack : for L2, 1/255 - 3/255, and for L infinite (norm = np.inf), 8/255

max iteration = 100 for cwl2


In [6]:
label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

In [7]:
## custom turn specialist function

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
#     print('Learning rate: ', lr)
    return lr

def turn_specialist(model : Model, path : str,
        x_tr: np.ndarray | None = None,
        y_tr: np.ndarray | None = None,
        x_v: np.ndarray | None = None,
        y_v: np.ndarray | None = None,
        epochs: int = 21,
        learning_rate : float = 1e-3,
        batch_size: int = 128,
        save_each: bool = False,
        save_bests: int | None = None,
        verbose: int = 1,
    ):
        
        # build specialist network
        base = Model(inputs = model.inputs, outputs = model.layers[-2].output, name="base")
        x    = keras.Input(shape=base.input_shape[1:], name="in")
        y    = Dense(10, name="dense")(base(x)) # 10 can be changed to len(newtarget)
        z    = layers.Softmax(name="softmax")(y)
        specialist = Model(inputs = x, outputs = z)
        specialist.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        lr_scheduler = LearningRateScheduler(lr_schedule)
        
        opt = tf.keras.optimizers.Adam(learning_rate)




        # callbacks
        callbacks = [ModelCheckpoint(path, monitor="val_accuracy",
                                    save_best_only=True, verbose=verbose)]
        
        callbacks += [LearningRateScheduler(lr_schedule),
                        ReduceLROnPlateau(factor=np.sqrt(0.1), patience=5, min_lr=5e-7)]

        # fit 
        hist = specialist.fit(x_tr, y_tr, batch_size=batch_size,
                        validation_data=(x_v, y_v),
                        epochs=epochs, callbacks=callbacks, verbose=verbose)



        # ---------- summary ----------
        metric = "val_accuracy"
        best = np.max(hist.history[metric])
        first = hist.history[metric][0]
        print(f"best {metric} {best:.3f} (first {first:.3f})")
    


In [10]:
training_y_long=np.concatenate([y_train,y_train],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)

In [ ]:
## Structure varies, adversarial is from ResNet56v1, PGD method, untargeted , L2 norm, 216/255 epsilon perturbation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
n = 9
v = 2
# ##
# training_y_long=np.concatenate([y_train,y_train],axis=0)
# training_y_long = keras.utils.to_categorical(training_y_long,10)
validating_y_long=np.concatenate([y_v,y_v],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,10)
# ##
# training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_untarget],axis=0)
# validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_untarget],axis=0)

# ##
# path = f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_untargeted.keras'
# print(f'fit starts {label_dict[(n,v)]}_naive_untargeted.keras')
# resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
# resnet_model.train(save_best_only=True,epochs=200)
# ##
# print(f'fit ends. Fine tunning.')
# specialist = turn_specialist(load_model(path),
#     f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_untargeted_fine.keras', 
#     x_train, y_train_categorical_10, 
#     x_val, y_val_categorical_10)
# #         ##
# #         print('more epochs on naive one')

# #         naive_one = load_model(path)
# #         datagen = ImageDataGenerator(  
# #         width_shift_range=0.1,
# #         height_shift_range=0.1,
# #         horizontal_flip=True
# #     )
# #         datagen.fit(training_x)

# #         batch_size = 128
# #         steps_per_epoch = len(training_x) // batch_size

# #         ##
# #         checkpoint_21 = ModelCheckpoint(
# #             filepath=f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_untargeted_21best.keras',
# #             monitor='val_accuracy',
# #             save_best_only=True,
# #             verbose=1
# #         )

# #         ##
# #         naive_one.fit(
# #                     x=datagen.flow(training_x, training_y_long, batch_size=batch_size),
# #                     epochs=21,
# #                     validation_data=(validating_x, validating_y_long),
# #                     steps_per_epoch=steps_per_epoch,
# #                     verbose=1,
# #                     callbacks=[checkpoint_21]
# #                     )

# ##
# training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_target_to_ll],axis=0)
# validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_target_to_ll],axis=0)

# ##
# path = f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_targeted.keras'
# print(f'fit starts {label_dict[(n,v)]}_naive_targeted.keras')
# resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
# resnet_model.train(save_best_only=True,epochs=200)
# ##
# print(f'fit ends. Fine tunning.')
# specialist = turn_specialist(load_model(path),
#     f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_targeted_fine.keras', 
#     x_train, y_train_categorical_10, 
#     x_val, y_val_categorical_10)
# #         ##
# #         print('more epochs on naive one')

# #         naive_one = load_model(path)
# #         datagen = ImageDataGenerator(  
# #         width_shift_range=0.1,
# #         height_shift_range=0.1,
# #         horizontal_flip=True
# #     )
# #         datagen.fit(training_x)

# #         batch_size = 128
# #         steps_per_epoch = len(training_x) // batch_size
# #         ##
# #         checkpoint_21 = ModelCheckpoint(
# #             filepath=f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_targeted_21best.keras',
# #             monitor='val_accuracy',
# #             save_best_only=True,
# #             verbose=1
# #         )
# #         ##
# #         naive_one.fit(
# #         x=datagen.flow(training_x, training_y_long, batch_size=batch_size),
# #         epochs=21,
# #         validation_data=(validating_x, validating_y_long),
# #         steps_per_epoch=steps_per_epoch,
# #         verbose=1,
# #         callbacks=[checkpoint_21]
# #     )
# ##
training_y_long=np.concatenate([y_train,y_train[:10001]],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,10)

##
training_x = np.concatenate([x_train,adv_x_tr_cw_by_resnet56v1_0_untarget],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_cw_by_resnet56v1_0_untarget],axis=0)
##
path = f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_cw.keras'
print(f'fit starts {label_dict[(n,v)]}_naive_cw.keras')
resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
resnet_model.train(save_best_only=True,epochs=200)
##
print(f'fit ends. Fine tunning.')
specialist = turn_specialist(load_model(path),
    f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_cw_fine.keras', 
    x_train, y_train_categorical_10, 
    x_val, y_val_categorical_10)
#         ##
#         print('more epochs on naive one')

#         naive_one = load_model(path)
#         datagen = ImageDataGenerator(  
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         horizontal_flip=True
#     )
#         datagen.fit(training_x)

#         batch_size = 128
#         steps_per_epoch = len(training_x) // batch_size

#         ##
#         checkpoint_21 = ModelCheckpoint(
#             filepath=f'adversarial_models/naive_method/{label_dict[(n,v)]}_naive_cw_21best.keras',
#             monitor='val_accuracy',
#             save_best_only=True,
#             verbose=1
#         )

#         ##
#         naive_one.fit(
#         x=datagen.flow(training_x, training_y_long, batch_size=batch_size),
#         epochs=21,
#         validation_data=(validating_x, validating_y_long),
#         steps_per_epoch=steps_per_epoch,
#         verbose=1,
#         callbacks=[checkpoint_21]
#     )


fit starts ResNet56v2_naive_cw.keras
Epoch 1/200
1563/1563 [==============================] - ETA: 0s - loss: 2.2697 - acc: 0.4899
Epoch 1: val_acc improved from -inf to 0.56970, saving model to adversarial_models/naive_method/ResNet56v2_naive_cw.keras
1563/1563 [==============================] - 56s 30ms/step - loss: 2.2697 - acc: 0.4899 - val_loss: 1.7143 - val_acc: 0.5697 - lr: 0.0010
Epoch 2/200
1563/1563 [==============================] - ETA: 0s - loss: 1.4873 - acc: 0.6322
Epoch 2: val_acc improved from 0.56970 to 0.60835, saving model to adversarial_models/naive_method/ResNet56v2_naive_cw.keras
1563/1563 [==============================] - 46s 29ms/step - loss: 1.4873 - acc: 0.6322 - val_loss: 1.4979 - val_acc: 0.6083 - lr: 0.0010
Epoch 3/200
1563/1563 [==============================] - ETA: 0s - loss: 1.2552 - acc: 0.6956
Epoch 3: val_acc did not improve from 0.60835
1563/1563 [==============================] - 45s 29ms/step - loss: 1.2552 - acc: 0.6956 - val_loss: 2.0523 - val

1563/1563 [==============================] - ETA: 0s - loss: 0.6010 - acc: 0.8742
Epoch 27: val_acc did not improve from 0.83185
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6010 - acc: 0.8742 - val_loss: 0.8294 - val_acc: 0.8105 - lr: 0.0010
Epoch 28/200
1563/1563 [==============================] - ETA: 0s - loss: 0.5969 - acc: 0.8764
Epoch 28: val_acc did not improve from 0.83185
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5969 - acc: 0.8764 - val_loss: 0.8464 - val_acc: 0.8155 - lr: 0.0010
Epoch 29/200
1563/1563 [==============================] - ETA: 0s - loss: 0.5943 - acc: 0.8747
Epoch 29: val_acc improved from 0.83185 to 0.83455, saving model to adversarial_models/naive_method/ResNet56v2_naive_cw.keras
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5943 - acc: 0.8747 - val_loss: 0.7215 - val_acc: 0.8346 - lr: 0.0010
Epoch 30/200
1562/1563 [============================>.] - ETA: 0s - loss: 0.5822 - acc: 0.8795

In [ ]:
## PGD method, targeted to least likely logit

training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_target_to_ll],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_target_to_ll],axis=0)

for n in [9]:
    for v in [1]:
        for label in [1,2,3]:
#         for label in [0,1,2,3]:
            path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras'
            print(f'fit starts {label_dict[(n,v)]}_ff_targeted-adv_{label}.keras')

            # Train full + fine model (will use all visible GPUs)
            resnet_model = ResNet(
                path,
                training_x, training_y_long,
                validating_x, validating_y_long,
                n=n, version=v
            )
            resnet_model.train(save_best_only=True, epochs=200)

            print(f'fit ends. Turn into specialist on original only')

            # Load best model and train specialist on original data
            specialist = turn_specialist(
                load_model(path),
                f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras',
                x_train, y_train_categorical_10,
                x_val, y_val_categorical_10
            )

fit starts ResNet56v1_ff_targeted-adv_1.keras


In [ ]:
## load pre-trained CIFAR10 ResNet models
folder = 'CIFAR10models/ResNet/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}


for f, model_ in loaded_models.items():
    base = f.replace('.keras','')
    parts = base.split('_')
    structure = label_dict[(int(parts[1]), int(parts[2][-1]))]
    idx = parts[-1][-1]
    

    path = f'CIFAR10models/more_tunned/{structure}_more_specialized_{idx}.keras'
    specialist = turn_specialist(
        model_,
        path,
        x_train, y_train_categorical_10,    
        x_val, y_val_categorical_10
        )

In [ ]:

training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_target_to_ll],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_target_to_ll],axis=0)

for n in [3,9]:
    for v in [1,2]:
        for label in [0,1,2,3]:
            path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras'
            print(f'fit starts {label_dict[(n,v)]}_ff_targeted-adv_{label}.keras')
            resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
            resnet_model.train(save_best_only=True,epochs=200)
            print(f'fit ends. Turn into specialist on original only')
            specialist = turn_specialist(load_model(path),
                f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras', 
                x_train, y_train_categorical_10, 
                x_val, y_val_categorical_10)




fit starts ResNet20v1_ff_targeted-adv_0.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.903 (first 0.844)
fit starts ResNet20v1_ff_targeted-adv_1.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.904 (first 0.822)
fit starts ResNet20v1_ff_targeted-adv_2.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.902 (first 0.800)
fit starts ResNet20v1_ff_targeted-adv_3.keras


In [ ]:
## Now adversarial is CWL2, max iteration = 100
training_x = np.concatenate([x_train,adv_x_tr_cw_by_resnet56v1_0_untarget],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_cw_by_resnet56v1_0_untarget],axis=0)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    for n in [3,9]:
        for v in [1,2]:
            for label in [0,1,2,3]:
                path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_cw-adv_{label}.keras'
                print(f'fit starts {label_dict[(n,v)]}_ff_cw-adv_{label}.keras')
                resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
                resnet_model.train(save_best_only=True,epochs=200)
                print(f'fit ends. Turn into specialist on original only')
                specialist = turn_specialist(load_model(path),
                    f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_cw-adv_{label}.keras', 
                    x_train, y_train_categorical_10, 
                    x_val, y_val_categorical_10)







In [9]:
folder = 'adversarial_models/full_and_fine/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

In [14]:
loaded_models.keys()

dict_keys(['ResNet20v1_ff_untargeted-adv_0.keras', 'ResNet20v1_ff_untargeted-adv_1.keras', 'ResNet20v1_ff_untargeted-adv_2.keras', 'ResNet20v1_ff_untargeted-adv_3.keras', 'ResNet20v2_ff_untargeted-adv_0.keras', 'ResNet20v2_ff_untargeted-adv_1.keras', 'ResNet20v2_ff_untargeted-adv_2.keras', 'ResNet20v2_ff_untargeted-adv_3.keras', 'ResNet56v1_ff_untargeted-adv_0.keras', 'ResNet56v1_ff_untargeted-adv_1.keras', 'ResNet56v1_ff_untargeted-adv_2.keras', 'ResNet56v1_ff_untargeted-adv_3.keras', 'ResNet56v2_ff_untargeted-adv_0.keras', 'ResNet56v2_ff_untargeted-adv_1.keras'])

In [18]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    attack_type = parts[2] + '_ff_pgd'  # untargeted
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    row = [f'{structure}, {attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


                            model   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1, untargeted-adv_pgd  0.8556  0.8384  0.8760  0.8520  0.855500
1  ResNet20v2, untargeted-adv_pgd  0.7979  0.7940  0.8147  0.8118  0.804600
2  ResNet56v1, untargeted-adv_pgd  0.8509  0.8291  0.8410  0.8339  0.838725
3  ResNet56v2, untargeted-adv_pgd  0.7930  0.8064     NaN     NaN  0.799700


In [19]:
folder = 'adversarial_models/specialized_from_full_and_fine/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

dict_keys(['classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet56v2_ff_untargeted-adv_0.keras'])


In [20]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[3]  # ResNet20v1, etc.
    attack_type = parts[5] + '_original_pgd'  # untargeted
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    row = [f'{structure}, {attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


                            model   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1, untargeted-adv_pgd  0.9029  0.8992  0.9010  0.9033  0.901600
1  ResNet20v2, untargeted-adv_pgd  0.9101  0.9060  0.9058  0.9052  0.906775
2  ResNet56v1, untargeted-adv_pgd  0.9095  0.9097  0.9079  0.9123  0.909850
3  ResNet56v2, untargeted-adv_pgd  0.9102     NaN     NaN     NaN  0.910200
